# Scraper para Plumas Libres

In [4]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time
import re

class PlumasLibresScraper:
    def __init__(self):
        self.base_url = "https://plumaslibres.com.mx/category/sociedad/page/{}/"
        self.articles_data = []
        self.lock = False
        self.article_pattern = re.compile(r'https://plumaslibres\.com\.mx/\d{4}/\d{2}/\d{2}/.+')
        
    def is_valid_article_url(self, url):
        """Verifica si la URL es de un artículo válido"""
        return bool(self.article_pattern.match(url)) and not any(x in url for x in ['/category/', '/author/'])
    
    def get_article_links(self, page_num):
        """Obtiene los enlaces a artículos válidos de una página"""
        url = self.base_url.format(page_num)
        try:
            response = requests.get(url, timeout=10)
            response.encoding = 'utf-8'  # Forzar codificación UTF-8
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            articles = soup.select('main.main div.col-sm-6 a[href]')
            valid_links = []
            
            for a in articles:
                href = a['href']
                if self.is_valid_article_url(href):
                    valid_links.append(href)
            
            return list(set(valid_links))
            
        except Exception as e:
            print(f"\nError al obtener enlaces de página {page_num}: {str(e)}")
            return []
    
    def scrape_article(self, url):
        """Extrae información de un artículo con manejo robusto de errores"""
        try:
            response = requests.get(url, timeout=10)
            response.encoding = 'utf-8'  # Forzar codificación UTF-8
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Verificar estructura básica del artículo
            article_body = soup.find('article')
            if not article_body:
                return False
                
            # Extraer título con manejo de error
            title_elem = soup.find('h1', class_='entry-title')
            title = title_elem.get_text(strip=True) if title_elem else "Sin título"
            
            # Extraer fecha
            date_elem = soup.find('time', class_='updated')
            date = date_elem.get_text(strip=True) if date_elem else "Sin fecha"
            
            # Extraer autor
            author = "Sin autor"
            share_div = soup.find('div', string=lambda t: t and "Compartir con WhatsApp" in t)
            if share_div:
                author_tag = share_div.find_next('p')
                if author_tag:
                    author = author_tag.get_text(strip=True).replace('Por ', '')
            
            # Extraer contenido
            content = []
            for p in article_body.find_all('p'):
                text = p.get_text(strip=True)
                if text and not any(x in text for x in ['Compartir con', 'Síguenos en']):
                    content.append(text)
            
            # Simulamos thread safety
            while self.lock:
                time.sleep(0.1)
            self.lock = True
            self.articles_data.append({
                'URL': url,
                'Título': title,
                'Artículo': '\n'.join(content),
                'Fecha': date,
                'Autor': author
            })
            self.lock = False
            
            return True
            
        except Exception as e:
            print(f"\nError al scrapear artículo {url}: {str(e)}")
            return False
    
    def scrape_page(self, page_num):
        """Procesa una página completa"""
        article_links = self.get_article_links(page_num)
        success_count = 0
        
        for link in article_links:
            if self.scrape_article(link):
                success_count += 1
        
        return (page_num, len(article_links)), success_count
    
    def scrape(self, start_page=1, end_page=47, max_workers=5):
        """Función principal con paralelización"""
        total_pages = end_page - start_page + 1
        total_articles = 0
        processed_articles = 0
        
        print(f"\nIniciando scraping de {total_pages} páginas (desde {start_page} hasta {end_page})")
        print(f"Usando {max_workers} workers para paralelización\n")
        
        start_time = time.time()
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Primera pasada: conteo total
            print("Calculando cantidad total de artículos...")
            future_to_page = {
                executor.submit(self.get_article_links, page_num): page_num 
                for page_num in range(start_page, end_page + 1)
            }
            
            for future in tqdm(as_completed(future_to_page), total=total_pages, desc="Analizando páginas"):
                article_links = future.result()
                total_articles += len(article_links)
            
            print(f"\nTotal de artículos válidos encontrados: {total_articles}\n")
            
            # Segunda pasada: scraping real
            print("Iniciando scraping paralelo de artículos...")
            future_to_page = {
                executor.submit(self.scrape_page, page_num): page_num 
                for page_num in range(start_page, end_page + 1)
            }
            
            with tqdm(total=total_articles, desc="Progreso") as pbar:
                for future in as_completed(future_to_page):
                    (page_num, page_articles), success_count = future.result()
                    pbar.update(page_articles)
                    processed_articles += success_count
        
        elapsed_time = time.time() - start_time
        print(f"\nScraping completado en {elapsed_time:.2f} segundos")
        print(f"Artículos procesados exitosamente: {processed_articles}/{total_articles}")
        print(f"Artículos fallidos: {total_articles - processed_articles}")
    
    def save_to_csv(self, filename="plumas_libres_sociedad.csv"):
        """Guarda los datos en CSV con codificación UTF-8"""
        with open(filename, 'w', encoding='utf-8-sig', newline='') as csvfile:  # Usamos utf-8-sig para BOM
            fieldnames = ['URL', 'titulo', 'articulo', 'fecha', 'autor']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(self.articles_data)
        
        print(f"\nDatos guardados en {filename} (Total: {len(self.articles_data)} artículos)")

if __name__ == "__main__":
    scraper = PlumasLibresScraper()
    
    # Configuración
    start_page = 1
    end_page = 47
    max_workers = 5  # Ajustar según necesidad
    
    scraper.scrape(start_page=start_page, end_page=end_page, max_workers=max_workers)
    scraper.save_to_csv()


Iniciando scraping de 47 páginas (desde 1 hasta 47)
Usando 5 workers para paralelización

Calculando cantidad total de artículos...


Analizando páginas: 100%|██████████| 47/47 [00:10<00:00,  4.50it/s]



Total de artículos válidos encontrados: 470

Iniciando scraping paralelo de artículos...


Progreso: 100%|██████████| 470/470 [01:05<00:00,  7.17it/s]


Scraping completado en 76.41 segundos
Artículos procesados exitosamente: 470/470
Artículos fallidos: 0


ValueError: dict contains fields not in fieldnames: 'Artículo', 'Fecha', 'Título', 'Autor'

In [6]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm  # Versión específica para Jupyter
import time
import re

class PlumasLibresScraper:
    def __init__(self):
        self.base_url = "https://plumaslibres.com.mx/category/sociedad/page/{}/"
        self.articles_data = []
        self.lock = False
        self.article_pattern = re.compile(r'https://plumaslibres\.com\.mx/\d{4}/\d{2}/\d{2}/.+')
        self.session = requests.Session()  # Mejor manejo de conexiones
        
    def is_valid_article_url(self, url):
        """Verifica si la URL es de un artículo válido"""
        return bool(self.article_pattern.match(url)) and not any(x in url for x in ['/category/', '/author/'])
    
    def get_article_links(self, page_num):
        """Obtiene los enlaces a artículos válidos de una página"""
        url = self.base_url.format(page_num)
        try:
            response = self.session.get(url, timeout=10)
            response.encoding = 'utf-8'
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            articles = soup.select('main.main div.col-sm-6 a[href]')
            valid_links = []
            
            for a in articles:
                href = a['href']
                if self.is_valid_article_url(href):
                    valid_links.append(href)
            
            return list(set(valid_links))
            
        except Exception as e:
            print(f"\nError al obtener enlaces de página {page_num}: {str(e)}")
            return []
    
    def scrape_article(self, url):
        """Extrae información de un artículo con manejo robusto de errores"""
        try:
            response = self.session.get(url, timeout=10)
            response.encoding = 'utf-8'
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            article_body = soup.find('article')
            if not article_body:
                return False
                
            # Extraer título
            title_elem = soup.find('h1', class_='entry-title')
            title = title_elem.get_text(strip=True) if title_elem else "Sin título"
            
            # Extraer fecha
            date_elem = soup.find('time', class_='updated')
            date = date_elem.get_text(strip=True) if date_elem else "Sin fecha"
            
            # Extraer autor
            author = "Sin autor"
            share_div = soup.find('div', string=lambda t: t and "Compartir con WhatsApp" in t)
            if share_div:
                author_tag = share_div.find_next('p')
                if author_tag:
                    author = author_tag.get_text(strip=True).replace('Por ', '')
            
            # Extraer contenido
            content = []
            for p in article_body.find_all('p'):
                text = p.get_text(strip=True)
                if text and not any(x in text for x in ['Compartir con', 'Síguenos en']):
                    content.append(text)
            
            # Thread safety
            while self.lock:
                time.sleep(0.1)
            self.lock = True
            self.articles_data.append({
                'url': url,
                'titulo': title,
                'articulo': '\n'.join(content),
                'fecha': date,
                'autor': author
            })
            self.lock = False
            
            return True
            
        except Exception as e:
            print(f"\nError al scrapear artículo {url}: {str(e)}")
            return False
    
    def scrape_page(self, page_num):
        """Procesa una página completa"""
        article_links = self.get_article_links(page_num)
        success_count = 0
        
        for link in article_links:
            if self.scrape_article(link):
                success_count += 1
        
        return (page_num, len(article_links)), success_count
    
    def scrape(self, start_page=1, end_page=47, max_workers=5):
        """Función principal con paralelización"""
        total_pages = end_page - start_page + 1
        total_articles = 0
        processed_articles = 0
        
        print(f"\nIniciando scraping de {total_pages} páginas (desde {start_page} hasta {end_page})")
        print(f"Usando {max_workers} workers para paralelización\n")
        
        start_time = time.time()
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Primera pasada: conteo total
            print("Calculando cantidad total de artículos...")
            future_to_page = {
                executor.submit(self.get_article_links, page_num): page_num 
                for page_num in range(start_page, end_page + 1)
            }
            
            for future in tqdm(as_completed(future_to_page), total=total_pages, desc="Analizando páginas"):
                article_links = future.result()
                total_articles += len(article_links)
            
            print(f"\nTotal de artículos válidos encontrados: {total_articles}\n")
            
            # Segunda pasada: scraping real
            print("Iniciando scraping paralelo de artículos...")
            future_to_page = {
                executor.submit(self.scrape_page, page_num): page_num 
                for page_num in range(start_page, end_page + 1)
            }
            
            with tqdm(total=total_articles, desc="Progreso") as pbar:
                for future in as_completed(future_to_page):
                    (page_num, page_articles), success_count = future.result()
                    pbar.update(page_articles)
                    processed_articles += success_count
        
        elapsed_time = time.time() - start_time
        print(f"\nScraping completado en {elapsed_time:.2f} segundos")
        print(f"Artículos procesados exitosamente: {processed_articles}/{total_articles}")
        print(f"Artículos fallidos: {total_articles - processed_articles}")
    
    def save_to_csv(self, filename="plumas_libres_sociedad.csv"):
        """Guarda los datos en CSV con codificación UTF-8"""
        if not self.articles_data:
            print("No hay datos para guardar")
            return
            
        try:
            with open(filename, 'w', encoding='utf-8-sig', newline='') as csvfile:
                fieldnames = ['url', 'titulo', 'articulo', 'fecha', 'autor']  # En minúsculas
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(self.articles_data)
            
            print(f"\nDatos guardados en {filename} (Total: {len(self.articles_data)} artículos)")
            return True
        except Exception as e:
            print(f"\nError al guardar el archivo CSV: {str(e)}")
            return False

# Ejecución en Jupyter Notebook
if __name__ == "__main__":
    scraper = PlumasLibresScraper()
    
    # Configuración (puedes ajustar estos valores)
    start_page = 1
    end_page = 47  # Recomiendo probar con pocas páginas primero
    max_workers = 7  # Más bajo para Jupyter
    
    # Ejecutar scraping
    scraper.scrape(start_page=start_page, end_page=end_page, max_workers=max_workers)
    
    # Guardar resultados
    scraper.save_to_csv()


Iniciando scraping de 47 páginas (desde 1 hasta 47)
Usando 7 workers para paralelización

Calculando cantidad total de artículos...


Analizando páginas:   0%|          | 0/47 [00:00<?, ?it/s]


Total de artículos válidos encontrados: 470

Iniciando scraping paralelo de artículos...


Progreso:   0%|          | 0/470 [00:00<?, ?it/s]


Scraping completado en 18.22 segundos
Artículos procesados exitosamente: 470/470
Artículos fallidos: 0

Datos guardados en plumas_libres_sociedad.csv (Total: 470 artículos)
